# Importing Libraries

In [1]:
from tensorflow.keras.models import model_from_json
from fer_model import get_fer_model
import tensorflow as tf
import tempfile 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from livelossplot import PlotLossesKerasTF
from tensorflow.keras.optimizers import Adam

import os
import zipfile

import tensorflow_model_optimization as tfmot
import pickle

In [2]:
from datetime import date
TIMESTAMP = str(date.today())

In [3]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

# Loading the CK+ Dataset

In [4]:
from data import load_ckplus
train_generator = load_ckplus("train")
test_generator = load_ckplus("test")

Found 924 images belonging to 8 classes.
Found 384 images belonging to 8 classes.


# Load Baseline Model

In [5]:
model = get_fer_model()
model.load_weights("weights/model_weights_2021-04-20.h5")

In [6]:
from evaluation import get_metrics
print("Baseline model performance:")
get_metrics(model, test_generator)

Baseline model performance:


{'size': 16512521,
 'acc': 0.7109375,
 'f_acc': 0.7,
 'm_acc': 0.7291666666666666,
 'f_acc_breakdown': [0.6666666666666666,
  0.5151515151515151,
  1.0,
  0.7083333333333334,
  0.5714285714285714,
  0.8666666666666667,
  0.26666666666666666,
  0.8888888888888888],
 'f_acc_balanced': 0.6854752886002886,
 'm_acc_breakdown': [0.5277777777777778,
  0.25,
  1.0,
  1.0,
  0.0,
  0.8333333333333334,
  0.3333333333333333,
  1.0],
 'm_acc_balanced': 0.6180555555555556}

# Apply Quantisation to the Baseline Model 

In [7]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
# This optimisation includes the quantisation 
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_tflite_model = converter.convert()

# Evaluate

In [8]:
from evaluation import get_metrics_quantised
metrics = get_metrics_quantised(quantized_tflite_model, test_generator)

Evaluated on 0 results so far.
Evaluated on 100 results so far.
Evaluated on 200 results so far.
Evaluated on 300 results so far.


In [9]:
print("Quantised model performance:")
metrics

Quantised model performance:


{'size': 4034807,
 'acc': 0.7109375,
 'f_acc': 0.7,
 'm_acc': 0.7291666666666666,
 'f_acc_breakdown': [0.6666666666666666,
  0.5151515151515151,
  1.0,
  0.7083333333333334,
  0.5714285714285714,
  0.8666666666666667,
  0.26666666666666666,
  0.8888888888888888],
 'f_acc_balanced': 0.6854752886002886,
 'm_acc_breakdown': [0.5277777777777778,
  0.25,
  1.0,
  1.0,
  0.0,
  0.8333333333333334,
  0.3333333333333333,
  1.0],
 'm_acc_balanced': 0.6180555555555556}

### Store results

In [10]:
with open("logs/quantised_model_metrics_%s.h5" % TIMESTAMP, 
          'wb') as quantised_model_metrics_file:
        pickle.dump(metrics, quantised_model_metrics_file)